This notebook aims to update the commercial and residential listings in accordance with their respective zones from the city zone map.

In [8]:
!pip install shapely

DEPRECATION: Loading egg at /Users/habib/miniconda3/envs/pytorch_m1_3.11/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [23]:
# print the zonal map data
import pandas as pd
import numpy as np
from shapely.geometry import MultiPolygon, Point
from shapely.wkt import loads as load_wkt

# read the data
zonal_data = pd.read_csv('../data/Zoning_Bylaw_Geographical_Data_20240528.csv')

# print the data
print(zonal_data.head(5))

       id  zoning               description    date_ext  \
0  169754      PS        Parks and Services  2024/05/27   
1  175633      CN  Neighbourhood Commercial  2024/05/27   
2  170985      CG        General Commercial  2024/05/27   
3  176327  RM h16  Medium Scale Residential  2024/05/27   
4  176677  RM h16  Medium Scale Residential  2024/05/27   

                                                 url  \
0  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
1  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
2  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
3  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
4  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   

                               geometry_multipolygon  
0  MULTIPOLYGON (((-113.46405030242478 53.4139561...  
1  MULTIPOLYGON (((-113.48544480430161 53.5005298...  
2  MULTIPOLYGON (((-113.41637166582207 53.5855382...  
3  MULTIPOLYGON (((-113.49564121741932 53.5688278...  
4  MULTIPOLYGON (((-113.377186670

In [24]:
# Function to check if a point lies within a multipolygon
def is_point_in_multipolygon(lat, lon):
    point = Point(lon, lat)  # Note: Shapely uses (lon, lat) order
    for _, row in zonal_data.iterrows():
        multipolygon_wkt = row['geometry_multipolygon']
        multipolygon = load_wkt(multipolygon_wkt)
        if multipolygon.contains(point):
            # return as row as json
            return row.to_dict()
    return {}

# Test the function with an example point
lat = 53.5005
lon = -113.4855

print({'lat': lat, 'lon': lon, 'zoning': is_point_in_multipolygon(lat, lon)})

{'lat': 53.5005, 'lon': -113.4855, 'zoning': {'id': 175633, 'zoning': 'CN', 'description': 'Neighbourhood Commercial', 'date_ext': '2024/05/27', 'url': 'http://webdocs.edmonton.ca/InfraPlan/zoningbylaw/bylaw_12800.htm', 'geometry_multipolygon': 'MULTIPOLYGON (((-113.48544480430161 53.500529804717324, -113.48575021281256 53.50071076505371, -113.48624647281675 53.50100695034535, -113.4862309463645 53.49987387916919, -113.48469211617721 53.49987363830927, -113.48469421839806 53.50005511947766, -113.4846925282687 53.50040641492085, -113.4852351676 53.50040558977945, -113.48544480430161 53.500529804717324)))'}}


In [25]:
# read the {} data and find its zone and update the data
# residential datas are in json format and in ../data/{}}/ folder they are slplited into multiple .js files

import pandas as pd

def update_zoning(listing_type):
    import json
    import os

    for file in os.listdir(f'../data/{listing_type}/'):
        with open(f'../data/{listing_type}/' + file) as f:
            data = json.load(f)
            listings = data['Results']
            
            for listing in listings:
                lat = listing['Property']['Address']['Latitude']
                lon = listing['Property']['Address']['Longitude']
                
                # Find the zoning for the listing
                zone = is_point_in_multipolygon(lat, lon)
                listing['zone'] = zone
                
                # append listing to a dataframe as a row
                df = pd.DataFrame([listing])
                
            
            # Save the updated data with _mapped suffix in the filename
            with open(f'../data/{listing_type}/' + file.replace('.js', '_zone_mapped.js'), 'w') as newfile:
                json.dump(data, newfile)
                newfile.close()
            
            f.close()
            
# Update the zoning for residential and commercial listings
update_zoning('residential')
update_zoning('commercial')

In [3]:
!pip install ace_tools

DEPRECATION: Loading egg at /Users/habib/miniconda3/envs/pytorch_m1_3.11/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement ace_tools (from versions: none)
ERROR: No matching distribution found for ace_tools


In [12]:
import pandas as pd
import json
import os

def update_zoning(listing_type):
    all_listings = []
    
    for file in os.listdir(f'../data/{listing_type}/'):
        if '_zone_mapped' in file:
            with open(f'../data/{listing_type}/' + file) as f:
                data = json.load(f)
                listings = data['Results']
                
                for listing in listings:
                    # add a listing type
                    listing['listing_type'] = listing_type
                    # Append listing to the all_listings list
                    all_listings.append(listing)
                
    # Create a DataFrame from all listings
    df = pd.DataFrame(all_listings)
    return df

# Update the zoning for residential and commercial listings
df_residential = update_zoning('residential')
df_commercial = update_zoning('commercial')

# Optionally, combine both dataframes into one
df_all = pd.concat([df_residential, df_commercial], ignore_index=True)

print(df_all.head(5))

# print the first first row as json
print(df_all.iloc[0].to_json())

# print the last row as json
print(df_all.iloc[-1].to_json())


         Id MlsNumber                                      PublicRemarks  \
0  26804622  E4384032  INVESTOR ALERT! Cheerful bungalow nestled in p...   
1  26828866  E4384868  Fantastic condo located in sought-after Belgra...   
2  26860468  E4386037  Beautiful bungalow home for sale in the desira...   
3  26665069  E4378735  Stunning, RIVER FACING, LARGE SIZED parcel of ...   
4  26539335  E4373981  Welcome to the community of Northmount! This u...   

                                            Building  \
0  {'BathroomTotal': '3', 'Bedrooms': '3', 'SizeI...   
1  {'BathroomTotal': '2', 'Bedrooms': '3', 'SizeI...   
2  {'BathroomTotal': '2', 'Bedrooms': '5', 'SizeI...   
3  {'Bedrooms': '0', 'StoriesTotal': '', 'Type': ...   
4  {'BathroomTotal': '4', 'Bedrooms': '5', 'SizeI...   

                                          Individual  \
0  [{'IndividualID': 2094464, 'Name': 'Samar Rang...   
1  [{'IndividualID': 2070038, 'Name': 'Dave Cousi...   
2  [{'IndividualID': 1999896, 'Name': 

In [11]:
import pandas as pd
import json
import os

def extract_listing_info(listing, listing_type):
    # Extract the necessary fields from the listing, handling missing values
    return {
        "price": listing["Property"].get("Price", ""),
        "type": listing["Property"].get("Type", ""),
        "address": listing["Property"]["Address"].get("AddressText", ""),
        "latitude": listing["Property"]["Address"].get("Latitude", ""),
        "longitude": listing["Property"]["Address"].get("Longitude", ""),
        "zone": [
            {
                "code": listing["zone"].get("zoning", "") if "zone" in listing else "",
                "description": listing["zone"].get("description", "") if "zone" in listing else ""
            }
        ],
        "photos": [photo.get("HighResPath", "") for photo in listing["Property"].get("Photo", [])],
        "parking": [parking.get("Name", "") for parking in listing["Property"].get("Parking", [])],
        "amenities_nearby": listing["Property"].get("AmmenitiesNearBy", "").split(", ") if listing["Property"].get("AmmenitiesNearBy") else [],
        "bedrooms": listing["Building"].get("Bedrooms", ""),
        "bathrooms": listing["Building"].get("BathroomTotal", ""),
        "size_interior": listing["Building"].get("SizeInterior", ""),
        "ownership_type": listing["Property"].get("OwnershipType", ""),
        "public_remarks": listing.get("PublicRemarks", ""),
        "postal_code": listing.get("PostalCode", ""),
        "historical_data": {
            "price_change_date": listing.get("PriceChangeDateUTC", ""),
            "photo_change_date": listing.get("PhotoChangeDateUTC", "")
        },
        "additional_features": listing.get("AdditionalFeatures", []),
        "listing_type": listing_type
    }

def update_zoning(listing_type):
    all_listings = []
    
    for file in os.listdir(f'../data/{listing_type}/'):
        if '_zone_mapped' in file:
            with open(f'../data/{listing_type}/' + file) as f:
                data = json.load(f)
                listings = data['Results']
                
                for listing in listings:
                    extracted_info = extract_listing_info(listing, listing_type)
                    all_listings.append(extracted_info)
                
    # Create a DataFrame from all listings
    df = pd.DataFrame(all_listings)
    return df

# Update the zoning for residential and commercial listings
df_residential = update_zoning('residential')
df_commercial = update_zoning('commercial')

# Optionally, combine both dataframes into one
df_all = pd.concat([df_residential, df_commercial], ignore_index=True)

# Display the dataframe to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="All Listings", dataframe=df_all)

print(df_all.head(5))

# Print the first row as JSON
print(df_all.iloc[0].to_json())

# save the data to a csv file
df_all.to_csv('../data/features.csv', index=False)

# save the data to a json file
df_all.to_json('../data/features.json', orient='records')

      price           type                                       address  \
0  $399,000  Single Family    8407 171 ST NW NW|Edmonton, Alberta T5T0C7   
1  $399,000  Single Family  #210 7327 118 ST NW|Edmonton, Alberta T6G1S5   
2  $399,000  Single Family       15717 87 AV NW|Edmonton, Alberta T5R4G7   
3  $399,000    Vacant Land       1115 147 AV NW|Edmonton, Alberta T5Y3Z6   
4  $399,000  Single Family       9116 137 AV NW|Edmonton, Alberta T5E1Y5   

           latitude          longitude  \
0  53.5173615338435       -113.6164052   
1        53.5096593  -113.534278415344   
2  53.5190815534664  -113.593025884656   
3  53.6100590786087         -113.36007   
4   53.599921200001       -113.4829512   

                                                zone  \
0  [{'code': 'RS', 'description': 'Small Scale Re...   
1  [{'code': 'RM h16', 'description': 'Medium Sca...   
2  [{'code': 'RS', 'description': 'Small Scale Re...   
3  [{'code': 'RS', 'description': 'Small Scale Re...   
4  [{'code